In [35]:
import pandas as pd

months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
years = ["2021", "2022", "2023"]
df = None

for year in years:
    print(f"loading {year}")
    for month in months:
        print(f"loading {month}")
        if year == "2019" and month not in [""]:continue
        if year == "2023" and month == "april":
            break
        data = pd.read_csv(f"data/apptx/{year}/{month}.csv")
        data = data.loc[data['Treatment Function Code'] == "C_999"]
        hospital_grouped = data.groupby('Provider Org Name')

        agg_functions = {}
        for col in data.columns:
            if data[col].dtype in [int, float]:
                agg_functions[col] = 'sum'
            else:
                agg_functions[col] = 'first'

        hospital_summed = hospital_grouped.agg(agg_functions)
        timecols = [col for col in hospital_summed if "SUM 1" in col]
        week_values = [int(col.split(" ")[1]) for col in timecols]
        renamed = {col: week_values[i] for i, col in enumerate(timecols)}
        out = hospital_summed.rename(columns=renamed)


        out["Period"] = out["Period"].str.slice(start=4)
        out = out.drop(columns=["Total","Patients with unknown clock start date", "Total All", "RTT Part Type","RTT Part Description","Treatment Function Code","Treatment Function Name","Commissioner Org Code","Commissioner Org Name","Commissioner Parent Name","Commissioner Parent Org Code","Provider Parent Org Code"])

        out.reset_index(drop=True, inplace=True)  # Reset index without adding existing column

        print(f"finished {year}.{month}")
        if df is None:
            df = out
        else:
            df = pd.concat((df,out), axis=0)

# creating total
df["Total"] = df.iloc[:,4:-1].sum(axis="columns")
df.head()


loading 2021
loading january
finished 2021.january
loading february
finished 2021.february
loading march
finished 2021.march
loading april
finished 2021.april
loading may
finished 2021.may
loading june
finished 2021.june
loading july
finished 2021.july
loading august
finished 2021.august
loading september
finished 2021.september
loading october
finished 2021.october
loading november
finished 2021.november
loading december
finished 2021.december
loading 2022
loading january
finished 2022.january
loading february
finished 2022.february
loading march
finished 2022.march
loading april
finished 2022.april
loading may
finished 2022.may
loading june
finished 2022.june
loading july
finished 2022.july
loading august
finished 2022.august
loading september
finished 2022.september
loading october
finished 2022.october
loading november
finished 2022.november
loading december
finished 2022.december
loading 2023
loading january
finished 2023.january
loading february
finished 2023.february
loading mar

,Period,Provider Parent Name,Provider Org Code,Provider Org Name,0,1,2,3,4,5,...,96,97,98,99,100,101,102,103,104,Total
0,JANUARY-2021,WEST YORKSHIRE AND HARROGATE HEALTH & CARE PAR...,RCF,AIREDALE NHS FOUNDATION TRUST,1082.0,1028.0,669.0,724.0,381.0,553.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13274.0
1,JANUARY-2021,CHESHIRE AND MERSEYSIDE STP,RBS,ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,487.0,376.0,347.0,483.0,314.0,475.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13600.0
2,JANUARY-2021,CAMBRIDGESHIRE AND PETERBOROUGH STP,NX5,ANGLIA COMMUNITY EYE SERVICE LTD,26.0,104.0,138.0,173.0,66.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1579.0
3,JANUARY-2021,SURREY HEARTLANDS HEALTH & CARE PARTNERSHIP (STP),RTK,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,1932.0,2368.0,2365.0,2388.0,1478.0,1770.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37077.0
4,JANUARY-2021,SURREY HEARTLANDS HEALTH & CARE PARTNERSHIP (STP),NVC01,ASHTEAD HOSPITAL,45.0,47.0,50.0,75.0,28.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1072.0


In [36]:
df.to_csv("data/all_hospital.csv")

In [ ]:
#converting data into percentages

In [39]:
percentage = df.columns[4:-1]
percentage_new = df
percentage_new[percentage] = df[percentage].div(df.Total, axis=0)*100
percentage_new.head()

,Period,Provider Parent Name,Provider Org Code,Provider Org Name,0,1,2,3,4,5,...,96,97,98,99,100,101,102,103,104,Total
0,JANUARY-2021,WEST YORKSHIRE AND HARROGATE HEALTH & CARE PAR...,RCF,AIREDALE NHS FOUNDATION TRUST,8.151273,7.744463,5.039928,5.454272,2.870273,4.166039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13274.0
1,JANUARY-2021,CHESHIRE AND MERSEYSIDE STP,RBS,ALDER HEY CHILDREN'S NHS FOUNDATION TRUST,3.580882,2.764706,2.551471,3.551471,2.308824,3.492647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13600.0
2,JANUARY-2021,CAMBRIDGESHIRE AND PETERBOROUGH STP,NX5,ANGLIA COMMUNITY EYE SERVICE LTD,1.646612,6.586447,8.739709,10.956301,4.179861,3.483217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1579.0
3,JANUARY-2021,SURREY HEARTLANDS HEALTH & CARE PARTNERSHIP (STP),RTK,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,5.210778,6.386709,6.378617,6.440651,3.986299,4.773849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37077.0
4,JANUARY-2021,SURREY HEARTLANDS HEALTH & CARE PARTNERSHIP (STP),NVC01,ASHTEAD HOSPITAL,4.197761,4.384328,4.664179,6.996269,2.611940,3.171642,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1072.0


In [40]:
percentage_new.to_csv("data/percentage_hospital.csv")